In [1]:
import sys
import math
import json
import pandas as pd
import cobra
import cplex
import optlang
from configparser import ConfigParser
config = ConfigParser()
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)
import cobrakbase
from IPython.core.display import HTML
from modelseedpy import MSPackageManager

cobrakbase 0.2.8


In [3]:
kbase_api = cobrakbase.KBaseAPI()
model= cobra.io.read_sbml_model("iML1515.xml")
model.solver = 'optlang-cplex'
gmm = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
pkgmgr = MSPackageManager.get_pkg_mgr(model)
pkgmgr.getpkg("KBaseMediaPkg").build_package(gmm)
biomass_objective = model.problem.Objective(
    1 * model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.flux_expression,
    direction='max')
model.objective = biomass_objective
solution = cobra.flux_analysis.pfba(model)
#Add the gene expression profile to the project notebook folder (Fatime)
expression_data = MSExpression.from_gene_feature_file("DifferentialExpression.tsv",None,False)
#Covert differential gene expression profile into differential reaction expression profile (Chris)
#Print this reaction profile and spot check values against Jill's manual numbers (Fatima,Jill)
#Add table of metabolomic differential expression to project notebook (Fatima)
#Need to load the metabolite differential profile data in terms of iML1515 IDs
#metabolite\tfoldchange
#Translate data into goals (Chris)
pkgmgr.getpkg("BilevelPkg").build_package(None,7)
pkgmgr.getpkg("ChangeOptPkg").build_package({
    "4HTHRK":{"flux":solution.fluxes["4HTHRK"]*1.2,"direction":"high","objcoef":1},
    "THRS":{"flux":solution.fluxes["THRS"]*1.2,"direction":"high","objcoef":1}
})
with open("multiomics.lp", 'w') as out:
    out.write(str(model.solver))
    out.close()
model.optimize()

4HTHRK
FOUND!


KeyError: 'bgoalc'